In [4]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import gensim.downloader as api
import pprint
import tqdm

pp = pprint.PrettyPrinter()

## train by gensim

In [2]:
print(api.info("text8"))
dataset = api.load("text8")

with open("dataset/text8.txt", "w") as f:
    for data in dataset:
        f.write(" ".join(data) + "\n")

{'num_records': 1701, 'record_format': 'list of str (tokens)', 'file_size': 33182058, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py', 'license': 'not found', 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.', 'checksum': '68799af40b6bda07dfa47a32612e5364', 'file_name': 'text8.gz', 'read_more': ['http://mattmahoney.net/dc/textdata.html'], 'parts': 1}


In [ ]:
dataset = open("dataset/text8.txt").read()
model = Word2Vec(dataset)

In [ ]:
print(model.wv.evaluate_word_pairs(datapath('wordsim353.tsv')))
print(model.wv.evaluate_word_pairs("word2vec/rw/rw_clean.txt")[1][0])
print(model.wv.evaluate_word_pairs(datapath('simlex999.txt')))

## train by word2vec diy

In [ ]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import gensim.downloader as api
import pprint
import tqdm

from word2vec.trainer import Word2VecTrainer
pp = pprint.PrettyPrinter()
%load_ext autoreload
%autoreload 2

wv = Word2VecTrainer("dataset/text8_300.txt", output_file="sgns.vec") # emb_dim=100, vocab=30000, SparseAdam, lr=0.001
# wv = Word2VecTrainer("dataset/text8.txt")
wv.train()

In [23]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

wv = KeyedVectors.load_word2vec_format("sgns.vec", binary=False)
vocab = list(wv.vocab.keys())
print("Loaded vocab size %i" % len(vocab))

Loaded vocab size 50000


In [27]:
print(wv.evaluate_word_pairs(datapath('wordsim353.tsv'))[1][0])
print(wv.evaluate_word_pairs("word2vec/rw/rw_clean.txt")[1][0])
print(wv.evaluate_word_pairs(datapath('simlex999.txt'))[1][0])

0.5027689131705056
0.2358323572771211
0.183232016465508
